# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [ ]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [ ]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [ ]:
from secrets import IEX_CLOUD_API_TOKEN

In [ ]:
symbol = 'AAPL'
api_url = f'https://api.iex.cloud/v1/data/core/advanced_stats/{symbol}?token={IEX_CLOUD_API_TOKEN}'
# print(api_url)
data = requests.get(api_url).json()
data

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [ ]:
data[0]['peRatio']

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [ ]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [ ]:
df = pd.DataFrame()
new_df1 = pd.DataFrame()
new_df2 = pd.DataFrame()

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/quote,advanced_stats/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    # print(batch_api_call_url)
    data = requests.get(batch_api_call_url).json()
    half = len(data)//2
    data1, data2 = data[:half], data[half:]

    for data_arr in data1:
        new_df = pd.DataFrame(
            [[
                data_arr['symbol'],
                data_arr['latestPrice']
            ]],columns=['Ticker', 'Price']
        )
        new_df1 = pd.concat([new_df1, new_df], ignore_index=True)

    for data_arr in data2:
        new_df = pd.DataFrame(
            [[
                data_arr['peRatio']
            ]],columns=['Price-to-Earnings Ratio']
        )
        new_df2 = pd.concat([new_df2, new_df], ignore_index=True)

df = new_df1.join(new_df2)
df

# 9.7s

## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [ ]:
df.sort_values('Price-to-Earnings Ratio', ascending=True, inplace=True)
df = df[df['Price-to-Earnings Ratio'] > 0]
df = df[:50]
df.reset_index(inplace=True, drop=True)
df

## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [ ]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [ ]:
portfolio_input()

You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [ ]:
new_df3 = pd.DataFrame()

position_size = float(portfolio_size) / len(df.index)

for i in df.index:
      
    new_df = pd.DataFrame(
            [[
                math.floor(position_size/df.loc[i, 'Price'])
            ]],columns=['Number of Shares to Buy']
        )
    new_df3 = pd.concat([new_df3, new_df], ignore_index=True)

df = df.join(new_df3)
df

## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [102]:
# rv_columns = [
#     'Ticker',
#     'Price',
#     'Number of Shares to Buy', 
#     'Price-to-Earnings Ratio',
#     'PE Percentile',
#     'Price-to-Book Ratio',
#     'PB Percentile',
#     'Price-to-Sales Ratio',
#     'PS Percentile',
#     'EV/EBITDA',
#     'EV/EBITDA Percentile',
#     'EV/GP',
#     'EV/GP Percentile',
#     'RV Score'
# ]

df = pd.DataFrame()
new_df1 = pd.DataFrame()
new_df2 = pd.DataFrame()

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/quote,advanced_stats/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    # print(batch_api_call_url)
    data = requests.get(batch_api_call_url).json()
    half = len(data)//2
    data1, data2 = data[:half], data[half:]

    for data_arr in data1:
        new_df = pd.DataFrame(
            [[
                data_arr['symbol'],
                data_arr['latestPrice']
            ]],columns=['Ticker', 'Price']
        )
        new_df1 = pd.concat([new_df1, new_df], ignore_index=True)

    for data_arr in data2:
        try:
            ev_to_ebitda = data_arr['enterpriseValue']/data_arr['EBITDA']
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = data_arr['enterpriseValue']/data_arr['grossProfit']
        except TypeError:
            ev_to_gross_profit = np.NaN

        new_df = pd.DataFrame(
            [[
                data_arr['peRatio'],
                data_arr['priceToBook'],
                data_arr['priceToSales'],
                ev_to_ebitda,
                ev_to_gross_profit
            ]],columns=[
                'Price-to-Earnings Ratio',
                'Price-to-Book Ratio',
                'Price-to-Sales Ratio',
                'EV/EBITDA',
                'EV/GP'
                ]
        )
        new_df2 = pd.concat([new_df2, new_df], ignore_index=True)

df = new_df1.join(new_df2)
df

# 10.5s

/tmp/ipykernel_1474/3168942819.py:64: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df2 = pd.concat([new_df2, new_df], ignore_index=True)


,Ticker,Price,Price-to-Earnings Ratio,Price-to-Book Ratio,Price-to-Sales Ratio,EV/EBITDA,EV/GP
0,A,132.03,31.201639,6.62,5.6600,24.574357,11.496402
1,AAL,15.13,12.051667,-1.90,0.1877,5.930057,0.667395
2,AAP,61.10,19.537996,1.37,0.3218,8.282314,1.086970
3,AAPL,182.52,27.929525,38.04,7.3100,21.930782,16.429871
4,ABBV,178.09,65.267892,30.37,5.7900,16.376178,10.652752
...,...,...,...,...,...,...,...
500,YUM,138.32,24.367210,-4.95,5.5000,20.194395,9.354476
501,ZBH,129.18,56.917833,2.16,3.7100,16.789999,6.172960
502,ZBRA,274.99,47.714481,4.65,3.0800,23.521751,7.633767
503,ZION,39.03,4.168888,1.19,1.1600,3.184057,1.201631


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [76]:
df[df.isnull().any(axis=1)]

,Ticker,Price,Price-to-Earnings Ratio,Price-to-Book Ratio,Price-to-Sales Ratio,EV/EBITDA,EV/GP
6,ABMD,381.020,NaN,NaN,NaN,NaN,NaN
28,ALXN,182.500,NaN,NaN,NaN,NaN,NaN
40,AON,315.320,NaN,NaN,NaN,NaN,NaN
48,ATVI,94.420,NaN,NaN,NaN,NaN,NaN
88,CERN,94.920,NaN,NaN,NaN,NaN,NaN
118,CTL,11.000,NaN,NaN,NaN,NaN,NaN
121,CTXS,103.900,NaN,NaN,NaN,NaN,NaN
136,DISCK,24.420,NaN,NaN,NaN,NaN,NaN
137,DISH,5.770,NaN,NaN,NaN,NaN,NaN
165,ETFC,49.260,NaN,NaN,NaN,NaN,NaN


In [83]:
print(df[(df['Price-to-Earnings Ratio'] < 0)])
print(df[(df['Price-to-Book Ratio'] < 0)])
print(df[(df['Price-to-Sales Ratio'] < 0)])
print(df[(df['EV/EBITDA'] < 0)])
print(df[(df['EV/GP'] < 0)])

    Ticker   Price  Price-to-Earnings Ratio  Price-to-Book Ratio  \
16     AES   16.00               -19.132258               5.5200   
19     AIV    7.30                -4.931365               2.0900   
26     ALL  159.13              -132.474423               2.6500   
55      BA  200.83               -55.145569              -7.1100   
63     BIO  337.99               -15.124171               1.1000   
84     CCL   15.23              -224.330932               2.4800   
124    CXO   65.60                -1.256963               1.6500   
149    DXC   21.32                -8.384521               1.2500   
163     ES   58.87               -46.549576               1.4500   
183    FIS   64.28                -1.569150               1.9600   
194    FTI   21.53              -274.940100               3.0600   
213    HAS   49.59               -12.352027               3.1300   
215    HBI    4.81               -95.270371               4.0300   
239   ILMN  133.00               -18.203015     

Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [92]:
df = df.dropna()
df.reset_index(inplace=True, drop=True)
df

,Ticker,Price,Price-to-Earnings Ratio,Price-to-Book Ratio,Price-to-Sales Ratio,EV/EBITDA,EV/GP
0,A,132.03,31.201639,6.62,5.6600,24.574357,11.496402
1,AAP,61.10,19.537996,1.37,0.3218,8.282314,1.086970
2,AAPL,182.52,27.929525,38.04,7.3100,21.930782,16.429871
3,ABBV,178.09,65.267892,30.37,5.7900,16.376178,10.652752
4,ABC,179.98,21.537842,53.04,0.1430,11.394933,4.604308
...,...,...,...,...,...,...,...
412,XYL,125.63,61.558133,3.05,4.4900,30.020360,12.569693
413,ZBH,129.18,56.917833,2.16,3.7100,16.789999,6.172960
414,ZBRA,274.99,47.714481,4.65,3.0800,23.521751,7.633767
415,ZION,39.03,4.168888,1.19,1.1600,3.184057,1.201631


Remove stocks with negative values

In [93]:
df = df[(df['Price-to-Earnings Ratio'] > 0) & (df['Price-to-Book Ratio'] > 0) & (df['Price-to-Sales Ratio'] > 0) & (df['EV/EBITDA'] > 0) & (df['EV/GP'] > 0)]
df.reset_index(inplace=True, drop=True)
df

,Ticker,Price,Price-to-Earnings Ratio,Price-to-Book Ratio,Price-to-Sales Ratio,EV/EBITDA,EV/GP
0,A,132.03,31.201639,6.62,5.6600,24.574357,11.496402
1,AAP,61.10,19.537996,1.37,0.3218,8.282314,1.086970
2,AAPL,182.52,27.929525,38.04,7.3100,21.930782,16.429871
3,ABBV,178.09,65.267892,30.37,5.7900,16.376178,10.652752
4,ABC,179.98,21.537842,53.04,0.1430,11.394933,4.604308
...,...,...,...,...,...,...,...
412,XYL,125.63,61.558133,3.05,4.4900,30.020360,12.569693
413,ZBH,129.18,56.917833,2.16,3.7100,16.789999,6.172960
414,ZBRA,274.99,47.714481,4.65,3.0800,23.521751,7.633767
415,ZION,39.03,4.168888,1.19,1.1600,3.184057,1.201631


Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [94]:
df[df.isnull().any(axis=1)]

,Ticker,Price,Price-to-Earnings Ratio,Price-to-Book Ratio,Price-to-Sales Ratio,EV/EBITDA,EV/GP


In [96]:
print(df[(df['Price-to-Earnings Ratio'] < 0)])
print(df[(df['Price-to-Book Ratio'] < 0)])
print(df[(df['Price-to-Sales Ratio'] < 0)])
print(df[(df['EV/EBITDA'] < 0)])
print(df[(df['EV/GP'] < 0)])

Empty DataFrame
Columns: [Ticker, Price, Price-to-Earnings Ratio, Price-to-Book Ratio, Price-to-Sales Ratio, EV/EBITDA, EV/GP]
Index: []
Empty DataFrame
Columns: [Ticker, Price, Price-to-Earnings Ratio, Price-to-Book Ratio, Price-to-Sales Ratio, EV/EBITDA, EV/GP]
Index: []
Empty DataFrame
Columns: [Ticker, Price, Price-to-Earnings Ratio, Price-to-Book Ratio, Price-to-Sales Ratio, EV/EBITDA, EV/GP]
Index: []
Empty DataFrame
Columns: [Ticker, Price, Price-to-Earnings Ratio, Price-to-Book Ratio, Price-to-Sales Ratio, EV/EBITDA, EV/GP]
Index: []
Empty DataFrame
Columns: [Ticker, Price, Price-to-Earnings Ratio, Price-to-Book Ratio, Price-to-Sales Ratio, EV/EBITDA, EV/GP]
Index: []


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [103]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

new_df4 = pd.DataFrame()
for i in range(0, len(df.index)):
    new_df3 = pd.DataFrame()
    for metric in metrics:
        new_df = pd.DataFrame(
                [[
                    stats.percentileofscore(df[f'{metric}'],df.loc[i,f'{metric}'], nan_policy='omit')/100,
                ]],
                columns=[metrics[metric]]
            )
        
        new_df3 = pd.concat([new_df3, new_df], axis="columns")
    
    new_df4 = pd.concat([new_df4, new_df3], ignore_index=True)

df = df.join(new_df4)
df

,Ticker,Price,Price-to-Earnings Ratio,Price-to-Book Ratio,Price-to-Sales Ratio,EV/EBITDA,EV/GP,PE Percentile,PB Percentile,PS Percentile,EV/EBITDA Percentile,EV/GP Percentile
0,A,132.03,31.201639,6.62,5.6600,24.574357,11.496402,0.718427,0.766529,0.800620,0.855372,0.791322
1,AAL,15.13,12.051667,-1.90,0.1877,5.930057,0.667395,0.236025,0.061983,0.012397,0.082645,0.018595
2,AAP,61.10,19.537996,1.37,0.3218,8.282314,1.086970,0.472050,0.194215,0.047521,0.198347,0.051653
3,AAPL,182.52,27.929525,38.04,7.3100,21.930782,16.429871,0.658385,0.969008,0.863636,0.805785,0.923554
4,ABBV,178.09,65.267892,30.37,5.7900,16.376178,10.652752,0.933747,0.962810,0.805785,0.601240,0.747934
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,138.32,24.367210,-4.95,5.5000,20.194395,9.354476,0.579710,0.055785,0.789256,0.764463,0.677686
501,ZBH,129.18,56.917833,2.16,3.7100,16.789999,6.172960,0.904762,0.381198,0.654959,0.630165,0.464876
502,ZBRA,274.99,47.714481,4.65,3.0800,23.521751,7.633767,0.875776,0.650826,0.592975,0.840909,0.572314
503,ZION,39.03,4.168888,1.19,1.1600,3.184057,1.201631,0.082816,0.157025,0.212810,0.024793,0.072314


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [104]:
from statistics import mean

new_df5 = pd.DataFrame()

for row in df.index:
    value_percentiles = []
    
    for metric in metrics:
        value_percentiles.append(df.loc[row, f'{metrics[metric]}'])
    
    new_df = pd.DataFrame(
            [[
                mean(value_percentiles)
            ]],
            columns=[
                'RV Score'
                ]
        )
    
    new_df5 = pd.concat([new_df5, new_df], ignore_index=True)

df = df.join(new_df5)
df

,Ticker,Price,Price-to-Earnings Ratio,Price-to-Book Ratio,Price-to-Sales Ratio,EV/EBITDA,EV/GP,PE Percentile,PB Percentile,PS Percentile,EV/EBITDA Percentile,EV/GP Percentile,RV Score
0,A,132.03,31.201639,6.62,5.6600,24.574357,11.496402,0.718427,0.766529,0.800620,0.855372,0.791322,0.786454
1,AAL,15.13,12.051667,-1.90,0.1877,5.930057,0.667395,0.236025,0.061983,0.012397,0.082645,0.018595,0.082329
2,AAP,61.10,19.537996,1.37,0.3218,8.282314,1.086970,0.472050,0.194215,0.047521,0.198347,0.051653,0.192757
3,AAPL,182.52,27.929525,38.04,7.3100,21.930782,16.429871,0.658385,0.969008,0.863636,0.805785,0.923554,0.844074
4,ABBV,178.09,65.267892,30.37,5.7900,16.376178,10.652752,0.933747,0.962810,0.805785,0.601240,0.747934,0.810303
...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,138.32,24.367210,-4.95,5.5000,20.194395,9.354476,0.579710,0.055785,0.789256,0.764463,0.677686,0.573380
501,ZBH,129.18,56.917833,2.16,3.7100,16.789999,6.172960,0.904762,0.381198,0.654959,0.630165,0.464876,0.607192
502,ZBRA,274.99,47.714481,4.65,3.0800,23.521751,7.633767,0.875776,0.650826,0.592975,0.840909,0.572314,0.706560
503,ZION,39.03,4.168888,1.19,1.1600,3.184057,1.201631,0.082816,0.157025,0.212810,0.024793,0.072314,0.109952


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [105]:
df.sort_values('RV Score', ascending=True, inplace=True)
df = df[:50]
df.reset_index(inplace=True, drop=True)
df

,Ticker,Price,Price-to-Earnings Ratio,Price-to-Book Ratio,Price-to-Sales Ratio,EV/EBITDA,EV/GP,PE Percentile,PB Percentile,PS Percentile,EV/EBITDA Percentile,EV/GP Percentile,RV Score
0,FRC,3.51,0.418421,0.0456,0.0934,0.270279,0.094878,0.080745,0.064050,0.002066,0.014463,0.004132,0.033091
1,GM,39.63,4.564976,0.7117,0.2662,1.647613,1.169939,0.086957,0.074380,0.018595,0.016529,0.061983,0.051689
2,C,55.93,8.703424,0.5634,0.7245,4.718687,0.762684,0.146998,0.066116,0.109504,0.049587,0.028926,0.080226
3,AAL,15.13,12.051667,-1.9000,0.1877,5.930057,0.667395,0.236025,0.061983,0.012397,0.082645,0.018595,0.082329
4,UNM,49.58,7.396372,0.9838,0.7666,4.475825,0.766635,0.111801,0.109504,0.119835,0.041322,0.030992,0.082691
5,UAL,45.51,5.217773,1.6900,0.2844,3.419762,0.528949,0.089027,0.272727,0.028926,0.026860,0.012397,0.085987
6,KSS,27.53,-21.464118,0.8126,0.1738,9.145677,1.243597,0.035197,0.078512,0.010331,0.245868,0.084711,0.090924
7,DXC,21.32,-8.384521,1.2500,0.2811,3.736878,2.147932,0.057971,0.176653,0.026860,0.028926,0.175620,0.093206
8,F,12.14,10.899416,1.1100,0.2689,2.673299,1.505392,0.198758,0.133264,0.020661,0.018595,0.109504,0.096157
9,VIAC,29.58,5.862651,0.9396,0.7205,6.649893,1.201202,0.099379,0.099174,0.107438,0.113636,0.070248,0.097975


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [106]:
portfolio_input()

In [107]:
new_df6 = pd.DataFrame()

position_size = float(portfolio_size) / len(df.index)

for i in range(0, len(df.index)):
      
    new_df = pd.DataFrame(
            [[
                math.floor(position_size/df.loc[i, 'Price'])
            ]],columns=['Number of Shares to Buy']
        )
    new_df6 = pd.concat([new_df6, new_df], ignore_index=True)

df = df.join(new_df6)
df

,Ticker,Price,Price-to-Earnings Ratio,Price-to-Book Ratio,Price-to-Sales Ratio,EV/EBITDA,EV/GP,PE Percentile,PB Percentile,PS Percentile,EV/EBITDA Percentile,EV/GP Percentile,RV Score,Number of Shares to Buy
0,FRC,3.51,0.418421,0.0456,0.0934,0.270279,0.094878,0.080745,0.064050,0.002066,0.014463,0.004132,0.033091,56980
1,GM,39.63,4.564976,0.7117,0.2662,1.647613,1.169939,0.086957,0.074380,0.018595,0.016529,0.061983,0.051689,5046
2,C,55.93,8.703424,0.5634,0.7245,4.718687,0.762684,0.146998,0.066116,0.109504,0.049587,0.028926,0.080226,3575
3,AAL,15.13,12.051667,-1.9000,0.1877,5.930057,0.667395,0.236025,0.061983,0.012397,0.082645,0.018595,0.082329,13218
4,UNM,49.58,7.396372,0.9838,0.7666,4.475825,0.766635,0.111801,0.109504,0.119835,0.041322,0.030992,0.082691,4033
5,UAL,45.51,5.217773,1.6900,0.2844,3.419762,0.528949,0.089027,0.272727,0.028926,0.026860,0.012397,0.085987,4394
6,KSS,27.53,-21.464118,0.8126,0.1738,9.145677,1.243597,0.035197,0.078512,0.010331,0.245868,0.084711,0.090924,7264
7,DXC,21.32,-8.384521,1.2500,0.2811,3.736878,2.147932,0.057971,0.176653,0.026860,0.028926,0.175620,0.093206,9380
8,F,12.14,10.899416,1.1100,0.2689,2.673299,1.505392,0.198758,0.133264,0.020661,0.018595,0.109504,0.096157,16474
9,VIAC,29.58,5.862651,0.9396,0.7205,6.649893,1.201202,0.099379,0.099174,0.107438,0.113636,0.070248,0.097975,6761


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [112]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [113]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.000000',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [114]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Price-to-Earnings Ratio', float_template],
                    'D': ['Price-to-Book Ratio', float_template],
                    'E': ['Price-to-Sales Ratio', float_template],
                    'F': ['EV/EBITDA', float_template],
                    'G': ['EV/GP', float_template],
                    'H': ['PE Percentile', percent_template],
                    'I': ['PB Percentile',percent_template],
                    'J': ['PS Percentile', percent_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP Percentile', percent_template],
                    'M': ['RV Score', percent_template],
                    'N': ['Number of Shares to Buy', integer_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [115]:
writer.close()